In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pathlib
import glob
import shutil
import os
import random
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

In [2]:
from keras.applications import ResNet101, ResNet152, ResNet50V2, ResNet101V2, ResNet152V2

In [3]:
%%capture
!pip install wandb -qqq
import wandb

In [4]:
from wandb.keras import WandbCallback

In [5]:
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
############################### caution : terminal commands ###########################################

#-----------------------------empty the datasets forlder before downloading the dataset ------------------------

%cd
%cd .keras/datasets/
!rm -r *

/root
/root/.keras/datasets


In [13]:
############################### download data from given url ###################################3

dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
data_dir = tf.keras.utils.get_file('nature_12K', origin=dataset_url, extract=True)


3816693760/3816687935 [==============================] - 67s 0us/step


In [14]:
#------------------------------caution : terminal commands --------------------------------------

%cd
%cd .keras/datasets/inaturalist_12K
%mv val test
!mkdir valid

/root
/root/.keras/datasets/inaturalist_12K


In [15]:
###################### split train data into validation set and training set ##########################

data_folder = '/root/.keras/datasets/inaturalist_12K'
os.chdir(data_folder)

folder_names = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia' ]
for i in range(0,10):
  source = data_folder + "/train/" +folder_names[i]  
  orig_files = os.listdir(source)
  chosen_indexes = random.sample(range(0, len(orig_files)-1), 100)

  destination = data_folder + "/valid/"
  os.chdir(destination)
  os.system('mkdir'+' '+str(folder_names[i]))
  destination = destination + folder_names[i]
  for j in range(0,100):
    shutil.move(   source +"/" + str(   orig_files[   chosen_indexes[j]  ]   )  , destination)


In [16]:
######################## Correcting the directory location #################################

#data_dir = '/root/.keras/datasets/nature_12K'

data_dir = data_dir.split('/')
data_dir.remove('nature_12K')
data_dir.append('inaturalist_12K')
data_dir = '/'.join(data_dir)


In [18]:
#----------------------------Taking the train data---------------------------------------------
train_data_dir_path = data_dir + '/train'
train_data_dir = pathlib.Path(train_data_dir_path)

#----------------------------Taking the validation data---------------------------------------
valid_data_dir_path = data_dir + '/valid'
valid_data_dir = pathlib.Path(valid_data_dir_path)

#-----------------------------Taking the test data----------------------------------------
test_data_dir_path = data_dir + '/test'
test_data_dir = pathlib.Path(test_data_dir_path)

In [19]:
#----------------------------Parameters for the images----------------------------------------
batch_size = 32
img_height = 180
img_width = 180

In [20]:
########################### converting the image data into dataset ##############################

#--------------------------training dataset ---------------------------------------
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode="categorical"
  )

#-------------------------validation dataset ----------------------------------
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  valid_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode="categorical"
  )

Found 9000 files belonging to 10 classes.
Found 999 files belonging to 10 classes.


In [21]:
######################################## function for using trained model ###############################


def get_desired_model(b_model, input_shape=(180,180,3), num_dense_neurons=1014,activation="relu",activation2="softmax",dropout=None, pooling="max",l_rate=0.0001,num_f=5): 
    
    if b_model=="Xception":
      base_model=Xception
    elif b_model=="InceptionV3":
      base_model=InceptionV3
    elif b_model=="ResNet50":
      base_model=ResNet50
    elif b_model=="MobileNet":
      base_model=MobileNet
    elif b_model == "ResNet101":
      base_model=ResNet101
    elif b_model == "ResNet152":
      base_model=ResNet152
    elif b_model == "ResNet50V2":
      base_model=ResNet50V2
    elif b_model == "ResNet101V2":
      base_model=ResNet101V2
    elif b_model == "ResNet152V2":
      base_model=ResNet152V2
    
    
      #--------------------------------- setting the trainable layers -------------------------------


    base = base_model(input_shape=input_shape,include_top=False,weights='imagenet') 

    for layer in base.layers[:num_f]:
      layer.trainable = False
  #------------------------------ perform pooling --------------------------------------
    if pooling == "avg": 
        x = GlobalAveragePooling2D()(base.output) 
    elif pooling == "max": 
        x = GlobalMaxPooling2D()(base.output) 

  #---------------------------- Flatten the output  --------------------------------------

    x = layers.Flatten()(base.output)


  #------------------------------- dropout ---------------------------------------------
    if dropout != 0: 
        x = layers.Dropout(dropout)(x) 

  #----------------------------- add fully connected layer ---------------------------------
    x = Dense(num_dense_neurons, activation=activation)(x) 

  #------------------------------- and a logistic layer -------------------------------------
    x = Dense(10, activation=activation2)(x) 

  #----------------------------------- get model ----------------------------------------------
    model = Model(inputs=base.input, outputs=x) 

  # #--------------------------------- setting the trainable layers -------------------------------
  #   for layer in base.layers:
  #       layer.trainable = False

 #------------------------------------compiling the model -----------------------------------------
    model.compile(loss='categorical_crossentropy', metrics=["accuracy"], optimizer=Adam(l_rate)) 
    
  #-------------------------------------return model -------------------------------------------------
    return model

#============================== end of get_desired_model function ========================================

In [22]:
#--------------------------- training -------------------------------------------
def train(model, train_ds, val_ds, epochs=5):
  history = model.fit(train_ds, validation_data = val_ds, epochs = epochs, callbacks=[WandbCallback()])
  return history

In [24]:
#--------------------------- Setting parameters for wandb -------------------------------------------

sweep_config={
              "method":"random",
              'metric' : {
                            'name' : 'val_accuracy',
                            'goal' : 'maximize',
                         },
          "parameters" : {
                            "base_model":{"values":["ResNet50", "Xception", "InceptionV3", "MobileNet"]},
                            "activation":{"values":['relu']},
                            "activation2":{"values":['softmax']},
                            "input_shape":{"values":[(180,180,3)]},
                            "dropout":{"values":[0,0.1,0.2]},
                            "num_dense_neuron":{"values":[256,512,1014]},
                            "l_rate":{"values":[0.0001,0.00005]},
                            "epoch": {"values": [5,10]},
                            "pooling":{"values":["max","avg", "no pooling"]},
                             "num_f":{"values":[-5,5,25]}
                         }
              }
sweep_id = wandb.sweep(sweep_config, project="pretrained")

Create sweep with ID: do6zfb98
Sweep URL: https://wandb.ai/pranayrajparisha/pretrained/sweeps/do6zfb98


In [26]:
#---------------------------  function for wandb -------------------------------------------

def run():

  wb = wandb.init()
  config = wb.config

  # sweep parameters
  base_model = config.base_model
  activation = config.activation
  activation2 = config.activation2
  dropout = config.dropout
  input_shape = config.input_shape
  num_dense_neuron = config.num_dense_neuron
  l_rate=config.l_rate
  epoch=config.epoch
  pooling = config.pooling
  num_f=config.num_f

  
  model = get_desired_model(base_model, input_shape, num_dense_neuron, activation, activation2, dropout, pooling, l_rate)
  history = train(model, train_ds, val_ds, epoch)

  return

In [ ]:
wandb.agent(sweep_id, run)

wandb: Agent Starting Run: 3dka0mj1 with config:
wandb: 	activation: relu
wandb: 	activation2: softmax
wandb: 	base_model: MobileNet
wandb: 	dropout: 0.2
wandb: 	epoch: 10
wandb: 	input_shape: [180, 180, 3]
wandb: 	l_rate: 0.0001
wandb: 	num_dense_neuron: 512
wandb: 	num_f: 5
wandb: 	pooling: avg
wandb: Currently logged in as: pranayrajparisha (use `wandb login --relogin` to force relogin)


17227776/17225924 [==============================] - 0s 0us/step
Epoch 1/10
282/282 [==============================] - 143s 379ms/step - loss: 2.4060 - accuracy: 0.2899 - val_loss: 1.8537 - val_accuracy: 0.3854
Epoch 2/10
282/282 [==============================] - 97s 335ms/step - loss: 1.1509 - accuracy: 0.6088 - val_loss: 1.8252 - val_accuracy: 0.4364
Epoch 3/10
282/282 [==============================] - 92s 323ms/step - loss: 0.6707 - accuracy: 0.7729 - val_loss: 2.1210 - val_accuracy: 0.4204
Epoch 4/10
282/282 [==============================] - 93s 325ms/step - loss: 0.3529 - accuracy: 0.8788 - val_loss: 2.1994 - val_accuracy: 0.4394
Epoch 5/10
282/282 [==============================] - 94s 327ms/step - loss: 0.2044 - accuracy: 0.9318 - val_loss: 2.4149 - val_accuracy: 0.4505
Epoch 6/10
282/282 [==============================] - 94s 329ms/step - loss: 0.1727 - accuracy: 0.9447 - val_loss: 2.7986 - val_accuracy: 0.4334
Epoch 7/10
282/282 [==============================] - 93s 326ms/

epoch,9
loss,0.11725
accuracy,0.95767
val_loss,2.96667
val_accuracy,0.47247
_runtime,988
_timestamp,1618510417
_step,9
best_val_loss,1.82525
best_epoch,1


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▃▂▁▁▁▁▁▁
accuracy,▁▄▆▇██████
val_loss,▁▁▃▃▅▇▅███
val_accuracy,▁▅▄▅▆▅▇▇██
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: updjc968 with config:
wandb: 	activation: relu
wandb: 	activation2: softmax
wandb: 	base_model: ResNet50
wandb: 	dropout: 0.2
wandb: 	epoch: 5
wandb: 	input_shape: [180, 180, 3]
wandb: 	l_rate: 0.0001
wandb: 	num_dense_neuron: 1014
wandb: 	num_f: -5
wandb: 	pooling: avg


...

Epoch 1/10
282/282 [==============================] - 138s 467ms/step - loss: 1.7728 - accuracy: 0.4861 - val_loss: 1.1350 - val_accuracy: 0.6406
Epoch 2/10
282/282 [==============================] - 132s 464ms/step - loss: 0.4428 - accuracy: 0.8535 - val_loss: 1.3006 - val_accuracy: 0.6527
Epoch 3/10
282/282 [==============================] - 132s 463ms/step - loss: 0.1343 - accuracy: 0.9545 - val_loss: 1.5149 - val_accuracy: 0.6617
Epoch 4/10
282/282 [==============================] - 131s 461ms/step - loss: 0.1231 - accuracy: 0.9588 - val_loss: 1.5358 - val_accuracy: 0.6476
Epoch 5/10
282/282 [==============================] - 132s 464ms/step - loss: 0.1061 - accuracy: 0.9646 - val_loss: 1.6897 - val_accuracy: 0.6306
Epoch 6/10
282/282 [==============================] - 132s 464ms/step - loss: 0.1109 - accuracy: 0.9656 - val_loss: 1.8485 - val_accuracy: 0.6286
Epoch 7/10
282/282 [==============================] - 132s 463ms/step - loss: 0.0988 - accuracy: 0.9685 - val_loss: 1.6587 -

epoch,9
loss,0.05508
accuracy,0.98133
val_loss,1.71603
val_accuracy,0.63463
_runtime,1336
_timestamp,1618538568
_step,9
best_val_loss,1.13496
best_epoch,0


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▁▁▁▁▁▁▁▁
accuracy,▁▆█▇██████
val_loss,▁▂▄▄▆▇▅▆█▆
val_accuracy,▄▆█▅▁▁▇▄▂▂
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: zic0qsyf with config:
wandb: 	activation: relu
wandb: 	activation2: softmax
wandb: 	base_model: MobileNet
wandb: 	dropout: 0
wandb: 	epoch: 10
wandb: 	input_shape: [180, 180, 3]
wandb: 	l_rate: 5e-05
wandb: 	num_dense_neuron: 256
wandb: 	num_f: -5
wandb: 	pooling: max


Epoch 1/10
282/282 [==============================] - 93s 315ms/step - loss: 2.3935 - accuracy: 0.2586 - val_loss: 1.8525 - val_accuracy: 0.3744
Epoch 2/10
282/282 [==============================] - 90s 314ms/step - loss: 1.2016 - accuracy: 0.5987 - val_loss: 1.8044 - val_accuracy: 0.4084
Epoch 3/10
282/282 [==============================] - 89s 310ms/step - loss: 0.6640 - accuracy: 0.7917 - val_loss: 1.8856 - val_accuracy: 0.4344
Epoch 4/10
282/282 [==============================] - 90s 313ms/step - loss: 0.3335 - accuracy: 0.9093 - val_loss: 2.0791 - val_accuracy: 0.4264
Epoch 5/10
123/282 [============>.................] - ETA: 46s - loss: 0.1726 - accuracy: 0.9580

In [ ]:
#https://wandb.ai/pranayrajparisha/pretrained/sweeps/do6zfb98?workspace=user-pranayrajparisha